In [1]:
include("TidalLoveNumbers.jl")
using .TidalLoveNumbers
using DoubleFloats
using PyPlot
using Statistics
using PyCall

@pyimport seaborn as sns
@pyimport matplotlib as mpl
@pyimport matplotlib.gridspec as gspec

PyPlot.isjulia_display[] = false;

PyPlot.matplotlib[:rc]("font",family="Arial", weight="medium", style="normal", size=12)
PyPlot.matplotlib[:rc]("axes",titlesize=14.5, labelsize=12, unicode_minus=false)
PyPlot.matplotlib[:rc]("xtick",labelsize=11.5)
PyPlot.matplotlib[:rc]("ytick",labelsize=11.5)
PyPlot.matplotlib[:rc]("pdf",fonttype=42)
PyPlot.matplotlib[:rc]("mathtext", fontset = "stix")

mpl.style.use("dark_background")

prec = TidalLoveNumbers.prec
precc = TidalLoveNumbers.precc

ComplexF64 (alias for Complex{Float64})

# Total Heating Rate

In [9]:
# Rotational and orbital parameters
ω = 5.31e-5     # Orbital frequency (2π / orbital period)
ecc = 0.05      # Orbtial eccentricity

# Internal structure:
# First element is the innermost layer, last element is the outermost layer
ρ = [3300, 3300, 3300, prec(3300)]  # Bulk density of each layer (kg m^-3)
r = [0,                             # Radii of each boundary (km)
     1,                             # Radius of CMB. If no core, set this to a 1km.
     200,                           
     230, 
     251.2] .* 1e3                  # Outer radius of solid surface 
μ = [60+0im, 60, 60, prec(60)].*1e9 # Shear modulus of each layer (Pa)
κ = [100e9, 100e9, 100e9, 100e9]    # Bulk modulus of each layer (set to a large number for incompressible material) (GPa)
η = [1e18, 1e18, 1e18, prec(1e18)]  # Shear viscosity of each layer (Pa s)  

μc = 1im * μ*ω ./ (1im*ω .+ μ./η)   # Complex shear modulus for a Maxwell material. Change this for different rheologies.

R = r[end]

r = expand_layers(r)
g = get_g(r, ρ);

In [ ]:
tidal_solution = calculate_y(r, ρ, g, μc, κ)       # Get "y-functions"

k2 = tidal_solution[5, end, end] - 1               # Get k2 Tidal Love Number 

Edot = get_bulk_heating(tidal_solution, ω, R, ecc) # Get total power output in watts
println("Total dissipation = ", Edot/1e9, " GW")

# Do sanity check/check for deviation from a homogeneous material
k2_analytical = 1.5 / (1 + 19/2 * μc[end] / (ρ[end]*g[end,end]*R))
println("Numerical error = ", abs(k2 - k2_analytical)/abs(k2_analytical)*100, "%")

Total dissipation = 0.09487087447078014 GW
Numerical error = 0.3380685920060883 %
